In [ ]:
import pandas as pd
import datetime

In [ ]:
#  1-1.Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. 
# Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).
data_class = pd.read_csv(r'''C:\Users\data\features.csv''', index_col='match_id')
y_fact=data_class['radiant_win']
x_fact=data_class.drop(['duration','tower_status_radiant','tower_status_dire','barracks_status_radiant',
                        'barracks_status_dire','radiant_win'],axis=1)

In [ ]:
#  1-2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает 
# число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, 
# и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.
deltaX_fact=x_fact.count()-len(x_fact)
deltaX_fact[deltaX_fact!=0]

In [ ]:
#  1-3. Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для 
# логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание.
# Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — 
# в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева.
# Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, 
# но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.
x_fact=x_fact.fillna(10000000)

In [ ]:
# 1-4 Какой столбец содержит целевую переменную? Запишите его название.

# radiant_win - целевая переменная

In [ ]:
#  1-5 Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на 
# имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не 
# забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без 
# перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга 
# (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум 
# протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли
# оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при 
# дальнейшем его увеличении?

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
resdfGBM=pd.DataFrame()
restimeGBM=pd.DataFrame()

def class_GBM(n_est):
    GBM_cl=GradientBoostingClassifier(n_estimators=n_est)
    my_cv=KFold(n_splits=5,shuffle=True)
    return cross_val_score(GBM_cl, x_fact, y_fact, cv=my_cv, scoring='roc_auc')

for i in range(10,100,10):
    start_time = datetime.datetime.now()
    resdfGBM[i]=class_GBM(i)
    restimeGBM.loc[i,'time']=(datetime.datetime.now() - start_time).total_seconds()

In [ ]:
В отчете по данному этапу вы должны ответить на следующие вопросы:

In [ ]:
1) Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?
first_blood_time              -19553
first_blood_team              -19553
first_blood_player1           -19553
first_blood_player2           -43987
radiant_bottle_time           -15691
radiant_courier_time            -692
radiant_flying_courier_time   -27479
radiant_first_ward_time        -1836
dire_bottle_time              -16143
dire_courier_time               -676
dire_flying_courier_time      -26098
dire_first_ward_time           -1826
first_blood_time,first_blood_team и остальные first_blood отсутсвуют значения т.к. не было убийств в течение 5 первых минут

In [ ]:
2) Как называется столбец, содержащий целевую переменную?
radiant_win - целевая переменная

In [ ]:
3) Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.
на 30 деревьях качество  0.690242
время на 30 деревьях  99.79 сек.

In [ ]:
4) Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?
сильно увеличивается время расчета при больще 30 деревьях несоизмеримая с приростом качетства
для ускорения расчета:
Предлагается накладывать ограничения на деревья например глубина
или ограничение на выборку( брать не всю выборку)

In [ ]:
#  2-1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью 
# кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр
# регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? 
# Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [68]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

data_class = pd.read_csv(r'''C:\Users\data\features.csv''', index_col='match_id')
y_fact=data_class['radiant_win']
x_fact1=data_class.drop(['duration','tower_status_radiant','tower_status_dire','barracks_status_radiant',
                        'barracks_status_dire','radiant_win'],axis=1)
x_fact1=x_fact1.fillna(10000000)
scaler = StandardScaler()
Scaled_x=scaler.fit_transform(x_fact1)
resdfLR=pd.DataFrame()
restimeLR=pd.DataFrame()
def class_LR(n_c):
    LR_cl=LogisticRegression(C=n_c)
    my_cv=KFold(n_splits=5,shuffle=True)
    return cross_val_score(LR_cl, Scaled_x, y_fact, cv=my_cv, scoring='roc_auc')
for i in range(1,10,1):
    int_s=pow(10,i-5)
    start_time = datetime.datetime.now()
    resdfLR[int_s]=class_LR(int_s)
    restimeLR.loc[i,'time']=(datetime.datetime.now() - start_time).total_seconds()


"\nscaler = StandardScaler()\nScaled_x=scaler.fit_transform(x_fact1)\nresdfLR=pd.DataFrame()\nrestimeLR=pd.DataFrame()\ndef class_LR(n_c):\n    LR_cl=LogisticRegression(C=n_c)\n    my_cv=KFold(n_splits=5,shuffle=True)\n    return cross_val_score(LR_cl, Scaled_x, y_fact, cv=my_cv, scoring='roc_auc')\nfor i in range(1,10,1):\n    int_s=pow(10,i-5)\n    start_time = datetime.datetime.now()\n    resdfLR[int_s]=class_LR(int_s)\n    restimeLR.loc[i,'time']=(datetime.datetime.now() - start_time).total_seconds()\n"

In [ ]:
#  2-2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей 
# идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero,
# d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой
# выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [64]:
y_fact=data_class['radiant_win']
x_fact2=data_class.drop(['duration','tower_status_radiant','tower_status_dire','barracks_status_radiant',
                        'barracks_status_dire','radiant_win','lobby_type', 'r1_hero',  'r2_hero', 'r3_hero',
                        'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'],axis=1)
x_fact2=x_fact2.fillna(10000000)
scaler = StandardScaler()
Scaled_x=scaler.fit_transform(x_fact2)
resdfLR=pd.DataFrame()
restimeLR=pd.DataFrame()
def class_LR(n_c):
    LR_cl=LogisticRegression(C=n_c)
    my_cv=KFold(n_splits=5,shuffle=True)
    return cross_val_score(LR_cl, Scaled_x, y_fact, cv=my_cv, scoring='roc_auc')
for i in range(1,10,1):
    start_time = datetime.datetime.now()
    resdfLR[i]=class_LR(i)
    restimeLR.loc[i,'time']=(datetime.datetime.now() - start_time).total_seconds()

In [108]:
resdfLR.mean().idxmax()

8

In [ ]:
#  2-3 На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли
# за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем 
# другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться 
# фукнция unique или value_counts).

In [ ]:
ss=data_class[['r1_hero',  'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']]
len(pd.unique(np.array(ss)))

In [ ]:
#  2-4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. 
# Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой 
# играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет
# данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [69]:
X_pick = np.zeros((x_fact1.shape[0], 112))

for i, match_id in enumerate(x_fact1.index):
    for p in range(5):
        X_pick[i, x_fact1.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, x_fact1.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [ ]:
#  2-5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации.
# Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [99]:
x_fact2=data_class.drop(['duration','tower_status_radiant','tower_status_dire','barracks_status_radiant',
                        'barracks_status_dire','radiant_win','lobby_type', 'r1_hero',  'r2_hero', 'r3_hero',
                        'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'],axis=1)
x_fact2=x_fact2.fillna(10000000)
scaler = StandardScaler()
x_fact3=scaler.fit_transform(x_fact2)
x_fact3=pd.DataFrame(x_fact3, index=x_fact2.index)
New_X=pd.concat([x_fact3, pd.DataFrame(X_pick,index=x_fact3.index)], axis=1)

resdfLR=pd.DataFrame()
restimeLR=pd.DataFrame()
def class_LR(n_c):
    LR_cl=LogisticRegression(C=n_c,random_state=42)
    my_cv=KFold(n_splits=5,shuffle=True)
    return cross_val_score(LR_cl, New_X, y_fact, cv=my_cv, scoring='roc_auc')
for i in range(1,10,1):
    int_s=pow(10,i-5)
    start_time = datetime.datetime.now()
    resdfLR[i]=class_LR(int_s)
    restimeLR.loc[i,'time']=(datetime.datetime.now() - start_time).total_seconds()

"\nresdfLR=pd.DataFrame()\nrestimeLR=pd.DataFrame()\ndef class_LR(n_c):\n    LR_cl=LogisticRegression(C=n_c,random_state=42)\n    my_cv=KFold(n_splits=5,shuffle=True)\n    return cross_val_score(LR_cl, New_X, y_fact, cv=my_cv, scoring='roc_auc')\nfor i in range(1,10,1):\n    int_s=pow(10,i-5)\n    start_time = datetime.datetime.now()\n    resdfLR[i]=class_LR(int_s)\n    restimeLR.loc[i,'time']=(datetime.datetime.now() - start_time).total_seconds()\n    "

In [ ]:
#  2-6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных 
# моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные
# — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [100]:
data_test = pd.read_csv(r'''C:\Users\data\features_test.csv''', index_col='match_id')
x_fact2=data_class.drop(['duration','tower_status_radiant','tower_status_dire','barracks_status_radiant',
                        'barracks_status_dire','radiant_win','lobby_type', 'r1_hero',  'r2_hero', 'r3_hero',
                        'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'],axis=1)
x_fact2=x_fact2.fillna(10000000)
scaler = StandardScaler()
x_fact3=scaler.fit_transform(x_fact2)
x_fact3=pd.DataFrame(x_fact3, index=x_fact2.index)

X_pick = np.zeros((x_fact1.shape[0], 112))

for i, match_id in enumerate(x_fact3.index):
    for p in range(5):
        X_pick[i, x_fact1.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, x_fact1.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
New_X_test=pd.concat([x_fact3, pd.DataFrame(X_pick,index=x_fact3.index)], axis=1)

In [103]:
LR_cl=LogisticRegression(C=1000,random_state=42)
my_cv=KFold(n_splits=5,shuffle=True)
LR_cl.fit(New_X,y_fact)
res_vect=LR_cl.predict_proba(New_X_test)

In [107]:
print(res_vect.min())
print(res_vect.max())

0.001169886388683693
0.9988301136113164


In [ ]:
В отчете по данному этапу вы должны ответить на следующие вопросы:


In [ ]:
#  1-1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством 
# градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению 
# с градиентным бустингом?
лучший параметр 0.0100    лучшее качество    0.715811
логистическая регрессия по сравнению с градиентным бустингом работает гораздо быстрее 


In [ ]:
# 1-2.Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)?
# Чем вы можете объяснить это изменение?
лучший параметр 0.0100  лучшее качество  0.715803
без категориальных признаков качество не изменилось,  т.к. категориальные признаки значительно не влияли на класификацию

In [ ]:
# 1-3. Сколько различных идентификаторов героев существует в данной игре?
108 различных идентификаторов героев существует, макимальный номер 112

In [ ]:
# 1-4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с 
# предыдущим вариантом? Чем вы можете это объяснить?
лучший параметр 1000  roc_auc 0.751952
качество значительно улучшилось(~0.04), так как категориальные признаки преобразованы в матрицу. В других экпериментах эти данные 
отсутсвовали, либо учитывались, как значения индефикатора героя, что не показательно.

In [ ]:
# 1-5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?
мин: 0.001169886388683693
макс: 0.9988301136113164